<a href="https://colab.research.google.com/github/KingAbdulRehman/LangChain/blob/main/LangChain_ToolCalling/LangChain_Tool_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import Packages

In [1]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.0 MB/s eta 0:00:00


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from google.colab import userdata

# Setup API Keys

In [3]:
if "GOOGLE_API_KEY" not in os.environ:
  os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

if "GOOGLE_CUSTOM_SEARCH_API" not in os.environ:
  os.environ["GOOGLE_CUSTOM_SEARCH_API"] = userdata.get("GOOGLE_CUSTOM_SEARCH_API")

# Setup LLM

In [ ]:
llm = ChatGoogleGenerativeAI(
  model="gemini-2.0-flash-exp",
  max_tokens=1000,
  max_retries=3,
  top_k=2,
  temperature=0.7
)

In [ ]:
# test llm
print(llm.invoke("Hello").content)

Hi there! How can I help you today?


# Create Search Tool